In [2]:
import numpy as np
import pandas as pd
# help(sr1.fillna)
# help(df1.fillna)
# help(pd.DataFrame.fillna)
# help(pd.Series.fillna)

### 缺失值

In [ ]:
"""注意dataframe中isnull,dropna,fillna的判断结果与bool判断的区别"""
df1 = pd.DataFrame([['',0],[True,False,],[None, np.nan,]])
sr1 = pd.Series(['',0,None,np.nan])
df1.isnull()
df1.notnull()
sr1.isnull()
sr1.notnull()
pd.isnull(sr1)
pd.notnull(sr1)
print(df1.applymap(bool))
print(pd.isnull(df1))
pd.notnull(df1)
df1.fillna('asdf')
df1.fillna({0:'a',1:'b',})
sr1.fillna('a')

In [ ]:
"""保留缺失值的行(np.nan或None)"""
df1[[any(i[1]) for i in df1.isnull().iterrows()]]
"""剔除缺失值的行(np.nan或None)"""
df1.dropna(axis=1,how='all',thresh=1) # how可取all或any,thresh只能用int
sr1.dropna()

### merge

In [ ]:
"""merge字段有重复时，会进行笛卡尔积"""
d1 = pd.DataFrame([['a','p'],['a','q']])
d2 = pd.DataFrame([['a','s'],['a','t']])
pd.merge(d1,d2,on=[0], how='outer') # how可选inner, outer, left, right
pd.merge(df1,df2,left_on=[,,],right_on=[,,],suffixes=['l','r'])
"""left_index的层次数要与right_on个数相等，不要求标签名匹配。匹配后的索引列成为value的一部分，匹配后的列名以right_on为准"""
pd.merge(frame1,frame2,left_index=True,right_on=[,,],sort=False,copy=True)
"""两边层次索引要一模一样，包括索引列的标签也要一样。匹配后，索引列仍然为索引列"""
pd.merge(frame1,frame2,left_index=True,right_index=True)

### concat

In [ ]:
"""df的"""
d1 = pd.DataFrame([['a','p'],['a','q']])
d2 = pd.DataFrame([['a','s'],['a','t']])
"""keys与frame个数一一对应，并成为层次化索引"""
pd.concat([d1,d2,],axis=0,keys=['x','y',])
"""传入字典，键会被当做keys选项的值，同上"""
pd.concat({'key1':d1,'key2':d2},axis=1)
"""不保留连接轴索引，产生一组新索引range(total_length)"""
pd.concat([d1,d2,],ignore_index=True)
"""sr的"""
s1 = pd.Series(list('abc'))
s2 = pd.Series(list('hij'), index=range(1,4))
"""纵向拼接返回series，keys用于层次索引"""
pd.concat([s1,s1,],axis=0,keys=['x','y',])
"""横向拼接返回dataframe, 按索引对齐，每个key对应一个列名，join可选inner,outer"""
pd.concat([s1,s2,], axis=1, keys=['x','y',], join='outer')
"""横向拼接返回dataframe, 指定索引"""
pd.concat([s1,s2,],axis=1,join_axes=[[1,3,5]])

### combine_first

In [ ]:
"""规则同下"""
d1.combine_first(d2)
"""sr的按索引合并，sr1索引有的以sr1为准, sr1索引没有的sr2加入，sr1中的None和np.nan会被sr覆盖"""
s1 = pd.Series([0,'',True,False,None,np.nan,np.nan])
s2 = pd.Series(list('hijklm'))
s1.combine_first(s2)

### replace方法

In [77]:
"""replace时候的Falset和0等价，True和1等价"""
df1 = pd.DataFrame([[1,0],[True,False,],[None, np.nan,],['',2]])
df1.replace('','space')
df1.replace([False,True],'TrueOrFalse')
df1.replace([0,1],['0','1'])
df1.replace([False,True],{False:'False',True:'True'})

,0,1
0,True,False
1,True,False
2,None,NaN
3,,2


### iterrows, iteritems, itertuples

In [48]:
df1 = pd.DataFrame(np.arange(6).reshape(2,3), columns=list('xyz'), index=list('ab'), dtype='int')  # 好像所有列只能指定一种类型
"""return iterator by rows, (index, Series)"""
list(df1.iterrows())  
"""return iterator by columns, (column, Series)"""
list(df1.iteritems())
"""return iterator by rows, tuple"""
list(df1.itertuples())